# Data LLM

In [12]:
# %run ../utils/db_loader.py ../data/processed/inventory.csv ../db/database.db INVENTORY_TABLE

Inserting rows:   0%|          | 0/750699 [00:00<?, ?row/s]

In [14]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect('../db/database.db')

# Load table into DataFrame
df = pd.read_sql_query("SELECT * FROM INVENTORY_TABLE LIMIT 10", conn)
display(df)

# Close connection
conn.close()


,Material Code,Material Description,Unit,Date,Open Stock,Material Issued,Material Received,Closing Stock,Prefix,Type,PDT,BFP,Price
0,400015,BAG; CANVAS TOOL 16'' LENGTH,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,90,0,750.0
1,400088,MACHINE; DRILL ELECTRIC WITH 13 BITS,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,90,0,1.0
2,400167,GOGGLE; FOR GAS WELDING,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,80,1,181.29
3,400176,"HAMMER; BALL PEEN, W/HANDLE 1LBS",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,120,0,1126.86
4,400177,"HAMMER; BALL PEEN, W/HANDLE 1.5LBS",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,120,0,1833.0
5,400178,"HAMMER; BALL PEEN, ENGG,W/HANDLE, 2LBS",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,90,0,1085.0
6,400180,HAMMER; MASON CHIESEL W/HANDLE 2-1/2LBS,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,150,0,6572.0
7,400205,"PLIER; INSULATED 8""",EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,60,1,1.0
8,400207,PLIER; NOSE COMBINATION 8'',EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,150,0,7905.0
9,400220,TAPE; STEEL FLEXIBLE 10' 3M,EA,2023-01-17,0.0,0.0,0.0,0.0,4,Tools,0,1,280.0


In [8]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../.env')

# Get the OpenAI API key from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key is None:
    raise ValueError("OpenAI API key not found in environment variables")

In [7]:
from openai import OpenAI

client = OpenAI(
  api_key=openai_api_key
  )

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  store=True,
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

print(completion.choices[0].message);

ChatCompletionMessage(content='Lines of code and dream,  \nWhispers of a mind unseen,  \nFuture intertwined.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
